In [ ]:
import os, glob , pandas as pd
from fastai import *
from fastai.vision import *

In [ ]:
train_image_path = glob.glob('../input/imaterialist-fashion-2020-fgvc7/train/*.jpg')

In [ ]:
with open('../input/imaterialist-fashion-2020-fgvc7/label_descriptions.json') as f:
    label_descriptions = json.load(f)
    
label_names = [x['name'] for x in label_descriptions['categories']]
num_categories = len(label_names)

In [ ]:
img_path = '../input/imaterialist-fashion-2020-fgvc7/train/'
label_path = '../input/fashion-segmentation-preprocessing/masks/'
get_label = lambda x: label_path + (str(x).split('/')[-1]).split('.')[0] + '.png'

In [ ]:
# Check whether the model work, or not before run on all data

# sample = train_image_path[1:501]
# # os.makedirs('./sample/')
# # os.makedirs('./masks/')
# for item in sample:
# #     print(item)
# #     break
#     filename = item.split('/')[-1]
#     maskname = filename.split('.')[0]
#     out = './sample/'+filename
#     srcmask = '../input/fashion-segmentation-preprocessing/masks/'+maskname+'.png'
#     outmask = './masks/'+maskname+'.png'
#     shutil.copyfile(item, out)
#     shutil.copyfile(srcmask, outmask)
    
# img_path = './sample/'
# label_path = './masks/'

In [ ]:
img = open_image(train_image_path[10])
mask = open_mask(get_label(train_image_path[10]))
_,axs = plt.subplots(1,3, figsize=(10,10))
img.show(ax=axs[0], title='no mask')
img.show(ax=axs[1], y=mask, title='masked')
mask.show(ax=axs[2], title='mask only', alpha=1.)

In [ ]:
codes = list(range(num_categories))
wd = 1e-2

In [ ]:
src_data = (SegmentationItemList.from_folder(img_path)\
            .split_by_rand_pct(valid_pct=0.1, seed=42)\
            .label_from_func(get_label, classes=codes))

In [ ]:
data = (src_data.transform(get_transforms(), size=224, tfm_y=True)\
        .databunch(bs=50).normalize(imagenet_stats)) #num_worker=2 should be used to speed up batch load

In [ ]:
data.show_batch(rows=3)

In [ ]:
data.label_list

In [ ]:
def acc_fashion(input, target):
    target = target.squeeze(1)
    mask = target != 28 # num_categories 
    return (input.argmax(dim=1)==target).float().mean()

In [ ]:
segment_learner = unet_learner(data, models.resnet34, metrics=acc_fashion, 
                               model_dir='/kaggle/working/models')

In [ ]:
# segment_learner.lr_find()
# segment_learner.recorder.plot()

In [ ]:
segment_learner.fit_one_cycle(4, max_lr=slice(1e-06, 1e-03))